## Exception handling

## Hierarchy of calls

In [ ]:
main()
    some_process()
        for filename in some_list:
            handle_file(filename)
                private_module.deal_with_file(filename)
                    private_module._helper_function(filename)
                       public_module.process_file(filename)
                           with open(filename) as fh:
                               pass

## Handling errors as return values

- Each function that fails returns some error indicator. None ? An object that has and attribute "error"?
- None would be bad as that cannot indicate different errors.
- Every called needs to check if the function returned error. If at any point we forget our system might run with hidden failures.

In [1]:
def some_function()
    result = do_something(filename)
    if result:
        do_something_else(result)
    else:
        return result


result = some_function()

SyntaxError: expected ':' (2823079409.py, line 1)

If we forget to check the result and pass it on, we might get some error in the code that is quite far from where the error actually happened


This can happen even if we don't pass the result around:


## Handling errors as exceptions

Only need to explicitly check for it at the level where we know what to do with the problem.
But: Do we want our pacemaker to stop totally after missing one beat? Probably not. Or better yet: not when it is in production.

## A simple exception

- When something goes wrong, Python throws (raises) an exception. For example, trying to divide a number by 0 won't work. If the exception is not handled, it will end the execution.

- In some programming languages we use the expression "throwing an exception" in other languages the expression is "raising an exception". I use the two expressions interchangeably.

- In the next simple example, Python will print the string before the division, then it will throw an exception, printing it to the standard error that is the screen by default. Then the script stops working and the string "after" is not printed.

In [ ]:
def div(a, b):
    print("before")
    print(a/b)
    print("after")

div(1, 0)

# before
# Traceback (most recent call last):
#   File "examples/exceptions/divide_by_zero.py", line 8, in <module>
#     div(1, 0)
#   File "examples/exceptions/divide_by_zero.py", line 5, in div
#     print(a/b)
# ZeroDivisionError: integer division or modulo by zero

## Prevention

- We might try to prevent the exceptions generated by the system, but even if succeed in preventing it, how do we indicate that there was an issue? For example with the input?

In [4]:
def div(a, b):
    if b == 0:
        # raise Exception("Cannot divide by zero")
        print("Cannot divide by zero")
        return None

    print("before")
    print(a/b)
    print("after")

div(1, 0)

Cannot divide by zero


## Working on a list

- In a slightly more interesting example we have a list of values. We would like to divide a number by each one of the values.

- As you can see one of the values is 0 which will generate and exception.

- The loop will finish early.

In [5]:
def div(a, b):
    print("dividing {} by {} is {}".format(a, b, a/b))

total = 100
values = [2, 5, 0, 4]

for val in values:
    div(total, val)

# dividing 100 by 2 is 50.0
# dividing 100 by 5 is 20.0
# Traceback (most recent call last):
# ...
# ZeroDivisionError: division by zero

dividing 100 by 2 is 50.0
dividing 100 by 5 is 20.0


ZeroDivisionError: division by zero

- We can't repair the case where the code tries to divide by 0, but it would be nice if we could get the rest of the results as well.

## Catch ZeroDivisionError exception

- For that, we'll wrap the critical part of the code in a "try" block. After the "try" block we need to provide a list of exception that are caught by this try-block.
- You could say something like "Try this code and let all the exceptions propagate, except of the ones I listed".
- As we saw in the previous example, the specific error is called ZeroDivisionError.

- If the specified exception occurs within the try: block, instead of the script ending, only the try block end and the except: block is executed.

In [7]:
import sys

def div(a, b):
    print("dividing {} by {} is {}".format(a, b, a/b))

total = 100
values = [2, 5, 0, 4]

for val in values:
    try:
        div(total, val)
    except ZeroDivisionError:
        print("Cannot divide by 0", file=sys.stderr)

# dividing 100 by 2 is 50.0
# dividing 100 by 5 is 20.0
# Cannot divide by 0
# dividing 100 by 4 is 25.0

dividing 100 by 2 is 50.0
dividing 100 by 5 is 20.0
dividing 100 by 4 is 25.0


Cannot divide by 0


## Module to open files and calculate something

- Of course in the previous example, it would be probably much easier if we just checked if the number was 0, before trying to divide with it. There are many other cases when this is not possible. For example it is impossible to check if open a file will succeed, without actually trying to open the file.

- In this example we open the file, read the first line which is a number and use that for division.

- When the `open()` fails, Python throws an FileNotFoundError exception.

In [8]:
def read_and_divide(filename):
    print("before " + filename)
    with open(filename, 'r') as fh:
        number = int(fh.readline())
        print(100 / number)
    print("after  " + filename)

## File for exception handling example

- If we have a list of files and we would like to make sure we process as many as possible without any problem caused in the middle, we can catch the exception.

- We have the following list of files. Notice that "two.txt" is missing and "zero.txt" has a 0 in it.

```
examples/exceptions/zero.txt
0

examples/exceptions/one.txt
1

File two.txt is missing on purpose.

examples/exceptions/three.txt
3
```


## Open files - exception

In [ ]:
import sys
import module

files = sys.argv[1:]

for filename in files:
    module.read_and_divide(filename)

# before one.txt
# 100.0
# after  one.txt
# before zero.txt
# Traceback (most recent call last):
# ...
# ZeroDivisionError: division by zero


## Handle divide by zero exception

In [ ]:
# Running this code will the ZeroDivisionError exception, but it will die with a FileNotFoundError exception.

import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except ZeroDivisionError:
        print(f"Cannot divide by 0 in file '{filename}'", file=sys.stderr)
    print('')

# before one.txt
# 100.0
# after  one.txt

# before zero.txt
# Cannot divide by 0 in file 'zero.txt'

# before two.txt
# FileNotFoundError: [Errno 2] No such file or directory: 'two.txt'


## Handle files - exception

In [ ]:
# We can add multiple "except" statement at the end of the "try" block and handle several exceptions. Each one in a different way.

import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except ZeroDivisionError:
        print(f"Cannot divide by 0 in file '{filename}'", file=sys.stderr)
    except FileNotFoundError:
        print(f"Cannot open file '{filename}'", file=sys.stderr)
    print('')

# before one.txt
# 100.0
# after  one.txt

# before zero.txt
# Cannot divide by 0 in file 'zero.txt'

# before two.txt
# Cannot open file 'two.txt'

# before three.txt
# 33.333333333333336
# after  three.txt

## Catch all the exceptions and show their type

In [ ]:
# We can also use the "except Exception" to catch all exceptions. In this case we might want to also print out the text and the type of the exception by ourselves.


import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except Exception as err:
        print(f"  There was a problem in '{filename}'", file=sys.stderr)
        print(f"  Text: {err}", file=sys.stderr)
        print(f"  Name: {type(err).__name__}", file=sys.stderr)
    print('')

# before one.txt
# 100.0
# after  one.txt

# before zero.txt
#   There was a problem in 'zero.txt'
#   Text: division by zero
#   Name: ZeroDivisionError

# before two.txt
#   There was a problem in 'two.txt'
#   Text: [Errno 2] No such file or directory: 'two.txt'
#   Name: FileNotFoundError

# before three.txt
# 33.333333333333336
# after  three.txt

## List exception types

In [ ]:
# We can list more than one exceptions to be caught one after the other in a single "except" statement.

except (ZeroDivisionError, FileNotFoundError):


import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except (ZeroDivisionError, FileNotFoundError) as err:
        print(f"We have a problem with file '{filename}'", file=sys.stderr)
        print(f"Exception type {err.__class__.__name__}", file=sys.stderr)
    print('')

# before one.txt
# 100.0
# after  one.txt

# before zero.txt
# We have a problem with file 'zero.txt'
# Exception type ZeroDivisionError

# before two.txt
# We have a problem with file 'two.txt'
# Exception type FileNotFoundError

# before three.txt
# 33.333333333333336
# after  three.txt

## Hierarchy of Exceptions

- There are many kinds of exceptions in Python and each module can define its own exception types as well. On this page you'll find the list and hierarchy of exceptions in Python.

https://docs.python.org/library/exceptions.html#exception-hierarchy


## Order of exception handling - bad

- Both exception are caught by the first except entry

In [ ]:
import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except Exception as err:
        print(f"General error {err}")
        print(f"Error class: {err.__class__.__name__}")
    except ZeroDivisionError:
        print("ZeroDivisionError")
        print(f"Cannot divide by 0 in file '{filename}'")
    print('')

# before one.txt
# 100.0
# after  one.txt

# before zero.txt
# General error division by zero
# Error class: ZeroDivisionError

# before two.txt
# General error [Errno 2] No such file or directory: 'two.txt'
# Error class: FileNotFoundError

# before three.txt
# 33.333333333333336
# after  three.txt




## Order of exception handling - good

- Always try to handle the more specific exceptions first

In [ ]:
import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except ZeroDivisionError:
        print("ZeroDivisionError")
        print(f"Cannot divide by 0 in file '{filename}'")
    except Exception as err:
        print(f"General error {err}")
        print(f"Error class: {err.__class__.__name__}")
    print('')

# before one.txt
# 100.0
# after  one.txt

# before zero.txt
# ZeroDivisionError
# Cannot divide by 0 in file 'zero.txt'

# before two.txt
# General error [Errno 2] No such file or directory: 'two.txt'
# Error class: FileNotFoundError

# before three.txt
# 33.333333333333336
# after  three.txt




## How to raise an exception

- As you create more and more complex applications you'll reach a point where you write a function, probably in a module, that needs to report some error condition. You can raise an exception in a simple way.

In [ ]:
def add_material(name, amount):
    if amount <= 0:
        raise Exception(f"Amount of {name} must be positive. {amount} was given.")
    print(f"Adding {name}: {amount}")

def main():
    things_to_add = (
        ("apple", 3),
        ("sugar", -1),
        ("banana", 2),
    )

    for name, amount in things_to_add:
        try:
            add_material(name, amount)
        except Exception as err:
            print(f"Exception: {err}")
            print("Type: " + type(err).__name__)

main()


# Adding apple: 3
# Exception: Amount of sugar must be positive. -1 was given.
# Type: Exception
# Adding banana: 2

## Raise ValueError exception

- You can be more specific with your error type and raise a ValueError.

In [ ]:
def add_material(name, amount):
    if amount <= 0:
        raise ValueError(f"Amount of {name} must be positive. {amount} was given.")
    print(f"Adding {name}: {amount}")

def main():
    things_to_add = (
        ("apple", 3),
        ("sugar", -1),
        ("banana", 2),
    )

    for name, amount in things_to_add:
        try:
            add_material(name, amount)
        except Exception as err:
            print(f"Exception: {err}")
            print("Type: " + type(err).__name__)

main()




# Adding apple: 3
# Exception: Amount of sugar must be positive. -1 was given.
# Type: ValueError
# Adding banana: 2

## Stack trace of exceptions

In [ ]:
import traceback

def bar():
    foo()

def foo():
    raise Exception("hi")

def main():
    try:
        bar()
    except Exception as err:
        track = traceback.format_exc()
        print("The caught:\n")
        print(track)

    print("---------------------")
    print("The original:\n")
    bar()


main()



# The caught:

# Traceback (most recent call last):
#   File "stack_trace.py", line 11, in main
#     bar()
#   File "stack_trace.py", line 4, in bar
#     foo()
#   File "stack_trace.py", line 7, in foo
#     raise Exception("hi")
# Exception: hi

# ---------------------
# The original:

# Traceback (most recent call last):
#   File "stack_trace.py", line 20, in <module>
#     main()
#   File "stack_trace.py", line 17, in main
#     bar()
#   File "stack_trace.py", line 4, in bar
#     foo()
#   File "stack_trace.py", line 7, in foo
#     raise Exception("hi")
# Exception: hi


## No need for exception to print Stack trace

In [ ]:
import traceback

def foo():
  bar()

def bar():
    #print(traceback.extract_stack())
    print(''.join(traceback.format_stack()))

foo()
print("done")

#   File "python/examples/other/print_stack_trace.py", line 10, in <module>
#     foo()
#   File "python/examples/other/print_stack_trace.py", line 4, in foo
#     bar()
#   File "python/examples/other/print_stack_trace.py", line 8, in bar
#     print(''.join(traceback.format_stack()))
#
# done

## Exercise: Exception int conversion

- In the earlier example we learned how to handle both ZeroDivisionError and FileNotFoundError exceptions. Now try this

```
 python handle_both_exceptions.py one.txt zero.txt two.txt text.txt three.txt
```


> before one.txt
>
> 100.0
> 
> after  one.txt
> 
> before zero.txt
> 
> Cannot divide by 0 in file 'zero.txt'
>
> before two.txt
>
> Cannot open file 'two.txt'
>
> before text.txt
>
```
Traceback (most recent call last):
  File "handle_both_exceptions.py", line 9, in <module>
    module.read_and_divide(filename)
  File "/home/gabor/work/slides/python/examples/exceptions/module.py", line 4, in read_and_divide
    number = int(fh.readline())
ValueError: invalid literal for int() with base 10: '3.14\n'
```

- This will raise a ValueError exception before handling file three.txt
- Fix it by capturing the specific exception.
- Fix by capturing "all other exceptions".


In [ ]:
import sys
import module

# python handle_both_exceptions.py one.txt zero.txt two.txt three.txt
files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except ZeroDivisionError:
        print("Cannot divide by 0 in file {}".format(filename))
    except IOError:
        print("Cannot open file {}".format(filename))
    except Exception as ex:
        print("Exception type {} {} in file {}".format(type(ex).__name__, ex, filename))




In [ ]:
import sys
import module

files = sys.argv[1:]

for filename in files:
    try:
        module.read_and_divide(filename)
    except ZeroDivisionError:
        print(f"Cannot divide by 0 in file '{filename}'")
    except FileNotFoundError:
        print(f"Cannot open file '{filename}'")
    except ValueError as err:
        print(f"ValueError {err} in file '{filename}'")

## Exercise: Raise Exception

- Write a function that expects a positive integer as its single parameter.
- Raise exception if the parameter is not a number.
- Raise a different exception if the parameter is not positive.
- Raise a different exception if the parameter is not whole number.

In [ ]:
def positive(num):
   if type(num).__name__ == 'float':
       raise Exception("The given parameter {} is a float and not an int.".format(num))

   if type(num).__name__ != 'int':
       raise Exception("The given parameter {} is of type {} and not int.".format(num, type(num).__name__))

   if num < 0:
       raise Exception("The given number {} is not positive.".format(num))

for val in [14, 24.3, "hi", -10]:
   print(val)
   print(type(val).__name__)
   try:
       positive(val)
   except Exception as ex:
       print("Exception: {}".format(ex))